# سیمینٹک کرنل میں ایجنٹ اسکریچ پیڈ کے ساتھ چیٹ ہسٹری کو کم کرنا

یہ نوٹ بک سیمینٹک کرنل کی چیٹ ہسٹری کو کم کرنے کی خصوصیت کو ایجنٹ اسکریچ پیڈ کے ساتھ استعمال کرنے کا طریقہ دکھاتی ہے تاکہ گفتگو کے دوران سیاق و سباق برقرار رکھا جا سکے۔ یہ لمبی گفتگو کو مؤثر طریقے سے سنبھالنے والے AI ایجنٹس بنانے کے لیے ضروری ہے، بغیر ٹوکن کی حد سے تجاوز کیے۔

## آپ کیا سیکھیں گے:
1. **چیٹ ہسٹری کو کم کرنا**: ٹوکن کے استعمال کو منظم کرنے کے لیے گفتگو کی ہسٹری کو خودکار طور پر خلاصہ کرنے کا طریقہ
2. **ایجنٹ اسکریچ پیڈ**: صارف کی ترجیحات اور مکمل شدہ کاموں کو ٹریک کرنے کے لیے ایک مستقل میموری سسٹم
3. **ٹوکن کے استعمال کی نگرانی**: ہسٹری کو کم کرنے کے ساتھ اور بغیر ٹوکن کے استعمال میں تبدیلی کو مانیٹر کرنا

## ضروریات:
- Azure OpenAI سیٹ اپ کے ساتھ ماحول کے متغیرات ترتیب دیے گئے ہوں
- پچھلے اسباق سے ایجنٹ کے بنیادی تصورات کی سمجھ


## مطلوبہ پیکجز درآمد کریں


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## ایجنٹ اسکریچ پیڈ کو سمجھنا

### ایجنٹ اسکریچ پیڈ کیا ہے؟

ایک **ایجنٹ اسکریچ پیڈ** ایک مستقل میموری سسٹم ہے جسے ایجنٹس درج ذیل کاموں کے لیے استعمال کرتے ہیں:
- **مکمل شدہ کاموں کا سراغ لگانا**: یہ ریکارڈ کرنا کہ صارف کے لیے کیا کیا جا چکا ہے
- **صارف کی ترجیحات کو محفوظ کرنا**: پسند، ناپسند اور ضروریات کو یاد رکھنا
- **سیاق و سباق کو برقرار رکھنا**: اہم معلومات کو گفتگو کے دوران دستیاب رکھنا
- **غیر ضروری تکرار کو کم کرنا**: بار بار ایک ہی سوالات پوچھنے سے بچنا

### یہ کیسے کام کرتا ہے:
1. **لکھنے کے عمل**: ایجنٹ نئی معلومات سیکھنے کے بعد اسکریچ پیڈ کو اپ ڈیٹ کرتا ہے
2. **پڑھنے کے عمل**: ایجنٹ فیصلے کرتے وقت اسکریچ پیڈ سے رجوع کرتا ہے
3. **مستقل مزاجی**: معلومات اس وقت بھی برقرار رہتی ہیں جب چیٹ کی تاریخ کم ہو جائے

اسے ایجنٹ کی ذاتی نوٹ بک سمجھیں جو گفتگو کی تاریخ کو مکمل کرتی ہے۔


## ماحول کی تشکیل


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## ایجنٹ اسکریچ پیڈ پلگ ان بنائیں

یہ پلگ ان ایجنٹ کو ایک مستقل اسکریچ پیڈ فائل پڑھنے اور لکھنے کی اجازت دیتا ہے۔


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## چیٹ ہسٹری ریڈیوسر کو شروع کریں

ChatHistorySummarizationReducer خودکار طور پر گفتگو کی ہسٹری کا خلاصہ تیار کرتا ہے جب یہ ایک مقررہ حد سے تجاوز کر جائے۔


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## چھٹیاں منصوبہ بندی کرنے والا ایجنٹ بنائیں

یہ ایجنٹ صارفین کو چھٹیاں منصوبہ بندی کرنے میں مدد کرے گا اور اس دوران اسکریچ پیڈ کے ذریعے سیاق و سباق کو برقرار رکھے گا۔


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

## ڈسپلے اور ٹوکن ٹریکنگ کے لیے مددگار فنکشنز


In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## چھٹیوں کی منصوبہ بندی کی گفتگو چلائیں

اب ایک مکمل گفتگو کے ذریعے چلتے ہیں جو درج ذیل چیزوں کو ظاہر کرتی ہے:
1. ابتدائی منصوبہ بندی کی درخواست
2. ترجیحات جمع کرنا
3. سفر کا منصوبہ بنانا
4. مقام کی تبدیلی
5. چیٹ کی تاریخ کو کم کرنا
6. اسکریچ پیڈ کا استعمال


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## نتائج کا تجزیہ

آئیے دیکھتے ہیں کہ ہماری گفتگو کے دوران کیا ہوا:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## اہم نکات

### 1. چیٹ ہسٹری میں کمی
- **خودکار عمل**: جب پیغامات کی تعداد حد سے تجاوز کرتی ہے تو کمی خود بخود ہوتی ہے
- **ٹوکین کی بچت**: خلاصہ کرنے کے بعد ٹوکین کے استعمال میں نمایاں کمی
- **سیاق و سباق کا تحفظ**: خلاصوں میں اہم معلومات محفوظ رہتی ہیں

### 2. ایجنٹ اسکریچ پیڈ کے فوائد
- **مستقل یادداشت**: صارف کی ترجیحات ہسٹری کی کمی کے بعد بھی برقرار رہتی ہیں
- **کام کی نگرانی**: ایجنٹ مکمل شدہ کام کا ریکارڈ رکھتا ہے
- **بہتر تجربہ**: ترجیحات کو دوبارہ دہرانے کی ضرورت نہیں

### 3. ٹوکین کے استعمال کے نمونے
- **لکیری اضافہ**: ہر پیغام کے ساتھ ٹوکین کی تعداد بڑھتی ہے
- **نمایاں کمی**: کمی کے بعد ٹوکین کی تعداد میں نمایاں کمی
- **پائیدار گفتگو**: حدود کے اندر طویل بات چیت کو ممکن بناتی ہے


## صفائی

اس ڈیمو کے دوران بنائی گئی اسکریچ پیڈ فائل کو صاف کریں:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# خلاصہ

مبارک ہو! آپ نے کامیابی سے ایک AI ایجنٹ تیار کیا ہے جو جدید کانٹیکسٹ مینجمنٹ کی صلاحیتوں سے لیس ہے:

## آپ نے کیا سیکھا:
- **چیٹ ہسٹری کی کمی**: گفتگو کو خودکار طور پر خلاصہ کرنا تاکہ ٹوکن کی حد کو منظم کیا جا سکے
- **ایجنٹ اسکریچ پیڈ**: صارف کی ترجیحات اور مکمل شدہ کاموں کے لیے مستقل یادداشت نافذ کرنا
- **ٹوکن مینجمنٹ**: طویل گفتگو میں ٹوکن کے استعمال کو ٹریک اور بہتر بنانا
- **کانٹیکسٹ کا تحفظ**: گفتگو کے خلاصے کے دوران اہم معلومات کو برقرار رکھنا

## حقیقی دنیا میں اطلاق:
- **کسٹمر سروس بوٹس**: سیشنز کے دوران صارف کی ترجیحات کو یاد رکھنا
- **ذاتی معاونین**: جاری منصوبوں اور صارف کی عادات کو ٹریک کرنا
- **تعلیمی اساتذہ**: طالب علم کی ترقی اور سیکھنے کی ترجیحات کو برقرار رکھنا
- **صحت کی دیکھ بھال کے معاونین**: مریض کی تاریخ کو محفوظ رکھنا جبکہ ٹوکن کی حد کا احترام کرنا

## اگلے اقدامات:
- زیادہ پیچیدہ اسکریچ پیڈ اسکیمیں نافذ کریں
- ملٹی یوزر منظرناموں کے لیے ڈیٹا بیس اسٹوریج شامل کریں
- مخصوص شعبوں کی ضروریات کے لیے کسٹم خلاصہ حکمت عملی تیار کریں
- سیمینٹک میموری سرچ کے لیے ویکٹر ڈیٹا بیس کے ساتھ امتزاج کریں
- ایسے ایجنٹس بنائیں جو دنوں بعد مکمل کانٹیکسٹ کے ساتھ گفتگو دوبارہ شروع کر سکیں



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے پوری کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا خامیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
